# Ethiopia Mapping Section

This Python Jupyter file is to calculate and build out the requirements for the Ethiopian design. It could be possible to do this in Excel as well; but this way we have an ability to reference and redesign as per the changing requirements.

First Step is to collect the information; I have a Github account under my username (johnmeye) which i will reference from the file so that anyone who uses Conda/Jupyter will be able to get the files. For any challenges reach out to me on teams or by email (johnmeye@cisco.com)

## Second Revision

This is the second revision of this file as there was a business case change from Vodacom. As a result I'm rewriting this to make it clearer and smoother to calculate the required output. 

## Inputs

The following are the inputs to this file received from Vodacom.
1. Site specifications 
2. Site locations
2. Business case 
3. Consumption assumption of users

## Outputs 

The following are the expected outputs of this file:
1. BoM for sites in catagories of:

    a. POC1
    
    b. POC2
    
    c. POC3
    
    d. Access
    
    e. Peering
    
    
2. BoM's will be in the correct format for CCW upload to allow for quick creation of the total costs.

In [83]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

#For the Map Plotting
import plotly
import plotly.express as px

#For the Inline display of figures.
from IPython.display import HTML
from IPython.display import IFrame

#For the Widgets
#Importing Widgets to allow for the changing of variables on the fly as questions are asked.
import ipywidgets as widgets 

#Fuzzy Matching:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

# Input Data 

## Geographical Data

This file below is from the Ethiopian files available from the database on the following site:
https://data.humdata.org/dataset/ethiopia-population-data-_-admin-level-0-3

This site has both topography and the Level 0-3 admin data on the population levels per county/province. Vodacom only provided at Admin 1 (Provincial level) for this RFQ; but we are able to go more in depth to try and work out if there is any additional information we can use to strengthen our position.

The following section will pull that information from my Github; so that you don't have to fetch it yourself. Then load it as a JSON file into the DB, which you can see are Polygon type files with GPS coordinates which mark out the different layers/levels in the country. 

If needed you can pull the information from the GeoJSON files as well, but i did include the boundaries data as a dataframe too. 
Example: counties3["features"][0]['properties'] #Just a sample on how to pull out specific information from the Counties json Files.

## Admin Level Data

As mentioned above; there is both Geo and Admin data; this information matches the information against some paramater; since the file is nicely structured according to standards we will stick to the humanitarian markings. 

Below i read the information from different levels into the variables for Admin1-3 so that we are able to use them to draw choropleth maps of the country. 

Once read into memory; it is possible to find matches against the specific parameters in both the GeoJSON and the Admin files. So i run a few sample commands to view what the data looks like. 

## Vodacom Sites Data
Vodacom has provided the Ethiopia site numbers, and the expected by year and by type, although the Vodacom breaks it down by height and rooftop; this might not be necessary from our point of view and should not impact the way we calculate this. 

For this we will need to figure out how to define rural/urban and so forth.

In [4]:
#*Geographical Data*
#Pull the Data I stored in my Github account for the analysis.

with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm1_csa_bofed_20190827.json') as response:
    counties1 = json.load(response)
    
with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm2_csa_bofed_20190827.json') as response:
    counties2 = json.load(response)
    
with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm3_csa_bofed_20190827.json') as response:
    counties3 = json.load(response)

#*Feature Data* 
#This data is available in the JSON files but its easier to manage from a tableset so i have pulled this below as well.
#Feature data is area, coordinates and naming conventions of each province/suburb/district in ethiopia.
    
Boundaries_Data1 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                  sheet_name='Admin1')

Boundaries_Data2 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                    sheet_name='Admin2')

Boundaries_Data3 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                    sheet_name='Admin3')


#*Admin Level Data*

Admin1 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin1')

Admin2 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin2')

Admin3 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin3')


### Site information input

The input file from Vodacom gives the sites by province and by year for the install. They have predefined what a site type is. 

In [38]:
Sites = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/Wakanda%20RFP_Updated%20Site%2BTraffic%20Inputs_04_Sep_2020.xlsx",
                   sheet_name='Site Nominals')
Sites['REGION'] = Sites['REGION'].str.title()
Sites['WOREDA'] = Sites['WOREDA'].str.title()


In [39]:
Sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10539 entries, 0 to 10538
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Longitude           10539 non-null  float64
 1   Latitude            10539 non-null  float64
 2   REGION              10539 non-null  object 
 3   WOREDA              10539 non-null  object 
 4   Woreda (alt name)   10539 non-null  object 
 5   Clutter Class Name  10539 non-null  object 
 6   Antenna  Height     10539 non-null  int64  
 7   TOWER TYPE          10539 non-null  object 
 8   SITE TYPE           10539 non-null  object 
 9   TECHNOLOGY          10539 non-null  object 
 10  On-Air Year         10539 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 905.8+ KB


In [80]:
NamedSites1 = pd.merge(Sites, Admin3[['admin3Name_en', 'admin3Pcode']],how='outer', left_on =['WOREDA'], right_on=['admin3Name_en'])
NaN_Sites2 = NamedSites1[NamedSites1['admin3Pcode'].isnull()]
NamedSites1 = NamedSites1[NamedSites1['admin3Pcode'].notna()]
NamedSites1 = NamedSites1[NamedSites1['REGION'].notna()]
NaN_Sites2 = NaN_Sites2.drop(['admin3Name_en', 'admin3Pcode'], axis=1)
NamedSites2 = pd.merge(NaN_Sites2, Admin3[['admin3Name_en', 'admin3Pcode']],how='left', left_on =['Woreda (alt name)'], right_on=['admin3Name_en'])
NaN_Sites2 = NamedSites2[NamedSites2['admin3Pcode'].isnull()]
NamedSites2 = NamedSites2[NamedSites2['admin3Pcode'].notna()]
NamedSites3 = pd.merge(NaN_Sites2, Admin3[['admin3Name_en', 'admin3Pcode']],how='left', left_on =['Woreda (alt name)'], right_on=['admin3Name_en'])

frames = [NamedSites1, NamedSites2]
result = pd.concat(frames)



In [90]:
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

str2Match = NaN_Sites2['WOREDA'].tolist()
strOptions =Admin3['admin3Name_en'].tolist()

name_match,ratio_match=checker(str2Match,strOptions)
df1 = pd.DataFrame()
df1['old_names']=pd.Series(str2Match)
df1['correct_names']=pd.Series(name_match)
df1['correct_ratio']=pd.Series(ratio_match)
df1.to_excel('matched_names.xlsx')

In [ ]:
NamedSites1.to_excel("output1.xlsx")
NamedSites2.to_excel("output2.xlsx")
NaN_Sites2.to_excel("output3.xlsx")
